# Temporal error evolution

> Authors: Alexander Geiss
>
> Abstract: Using AUX_MET data to determine the temporal evolution of L2B systematic and random errors for longer time periods.  

This notebook is currently "in progress" and covers the loading of the data for now.

## Load packages, modules and extensions

In [ ]:
# %load_ext blackcellmagic
# enable following line for interactive visualization backend for matplotlib
# %matplotlib widget
# print version info
%load_ext watermark
%watermark -i -v -p viresclient,pandas,xarray,matplotlib

In [ ]:
from viresclient import AeolusRequest
import numpy as np
import netCDF4 as nc
import xarray as xr
import pandas as pd
import os
# import multiprocessing as mp
# import matplotlib.pyplot as plt
# from matplotlib.collections import PolyCollection
# import matplotlib.cm as cm
# import matplotlib.colors as colors
# import cartopy.crs as ccrs

## Load AUX_MET and Aeolus L2B data
For a detailed description on how to load AUX_MET and L2B data, please see the corresponding notebooks.  
Computing long-term statistics of Aeolus' errors requires a different strategy for loading and handling large data sets with the available system resources. One option is to create requests on a monthly basis and save the retrieved data directly as netcdf data in the user space. Because some of the data within one month can be incomplete this would lead to a failed request for the whole month. Therefore, we reduce each request to half-month periods.
The retrieved and saved data can then be loaded with xarray and dask functionality to concatenate and process the files. Here it must be taken care of saved but invalid data which could be empty.

In [ ]:
# create start and end dates (here for 1 year of data)
dates = pd.date_range(start='2021-01-01', end='2022-01-01', freq='SMS').to_pydatetime()
start_dates = dates[:-1]
end_dates = dates[1:]

### Load L2B data
Define fields

In [ ]:
# Aeolus product
DATA_PRODUCT = "ALD_U_N_2B"

# define fields to retrieve
parameter_rayleigh = [
    "rayleigh_wind_result_start_time",
    "rayleigh_wind_result_stop_time",
    "rayleigh_wind_result_top_altitude",
    "rayleigh_wind_result_bottom_altitude",
    "rayleigh_wind_result_start_latitude",
    "rayleigh_wind_result_start_longitude",
    "rayleigh_wind_result_stop_latitude",
    "rayleigh_wind_result_stop_longitude",
    "rayleigh_wind_result_range_bin_number",
    "rayleigh_wind_result_wind_velocity",
    "rayleigh_wind_result_HLOS_error",
    "rayleigh_wind_result_los_azimuth",
    "rayleigh_wind_result_observation_type",
    "rayleigh_wind_result_validity_flag",
]

parameter_mie = [
    "mie_wind_result_start_time",
    "mie_wind_result_stop_time",
    "mie_wind_result_top_altitude",
    "mie_wind_result_bottom_altitude",
    "mie_wind_result_start_latitude",
    "mie_wind_result_start_longitude",
    "mie_wind_result_stop_latitude",
    "mie_wind_result_stop_longitude",
    "mie_wind_result_range_bin_number",
    "mie_wind_result_wind_velocity",
    "mie_wind_result_HLOS_error",
    "mie_wind_result_los_azimuth",
    "mie_wind_result_observation_type",
    "mie_wind_result_validity_flag",
]

Generate file names and paths for saving data periods as netCDF files.  

In [ ]:
L2B_rayleigh_filenames = [
    f"L2B_rayleigh_{start.strftime('%Y-%m-%d')}-{end.strftime('%Y-%m-%d')}.nc"
    for start, end in zip(start_dates, end_dates)
]
L2B_mie_filenames = [
    f"L2B_mie_{start.strftime('%Y-%m-%d')}-{end.strftime('%Y-%m-%d')}.nc"
    for start, end in zip(start_dates, end_dates)
]
user_folder = os.path.expanduser("~")
L2B_folder = os.path.join(user_folder, "files/L2B")
os.makedirs(L2B_folder, exist_ok=True)
L2B_rayleigh_file_paths = [
    os.path.join(L2B_folder, filename) for filename in L2B_rayleigh_filenames
]
L2B_mie_file_paths = [os.path.join(L2B_folder, filename) for filename in L2B_mie_filenames]

**Request L2B Rayleigh data**  
This can take some time depending on the requested time period.  
Here we repeat the get_between method for our list of half-months and save the retrieved (netCDF)-files to our user space right away using the to_file method.
The data for the requested period is only retrieved and saved if it is not yet available under the defined file path.
The list where the file paths are saved is then checked for files which are empty (have no groups in case of L2B data).

In [ ]:
# Create Aeolus request
request = AeolusRequest()
request.set_collection(DATA_PRODUCT)
request.set_fields(rayleigh_wind_fields=parameter_rayleigh)

# Set bounding box for region around Germany
request.set_bbox({"n": 60, "w": 0, "s": 45, "e": 20})

# create list which will hold the loaded Rayleigh data
L2B_rayleigh_file_paths_loaded = []
for measurement_start, measurement_stop, filename in zip(
    start_dates, end_dates, L2B_rayleigh_file_paths
):
    # check if file already exists
    if not os.path.isfile(filename):
        try:
            L2B_rayleigh_data = request.get_between(
                start_time=measurement_start,
                end_time=measurement_stop,
                filetype="nc",
                asynchronous=True,
            ).to_file(filename, overwrite=True)
            L2B_rayleigh_file_paths_loaded.append(filename)
        except Exception as e:
            print(f"No data for the period {measurement_start}-{measurement_stop} available")
            # context is printed here because it holds the true reason for the Exception
            print(e.__context__)
    else:
        L2B_rayleigh_file_paths_loaded.append(filename)

# Check file list for empty files
L2B_rayleigh_file_paths_valid = [i for i in L2B_rayleigh_file_paths_loaded if nc.Dataset(i).groups]

**Request L2B Mie data**  
This can take some time depending on the requested time period.

In [ ]:
# Create Aeolus request
request = AeolusRequest()
request.set_collection(DATA_PRODUCT)
request.set_fields(mie_wind_fields=parameter_mie)

# Set bounding box for region around Germany
request.set_bbox({"n": 60, "w": 0, "s": 45, "e": 20})

# create list which will hold the loaded Mie data
L2B_mie_file_paths_loaded = []
for measurement_start, measurement_stop, filename in zip(
    start_dates, end_dates, L2B_mie_file_paths
):
    # check if file already exists
    if not os.path.isfile(filename):
        try:
            L2B_mie_data = request.get_between(
                start_time=measurement_start,
                end_time=measurement_stop,
                filetype="nc",
                asynchronous=True,
            ).to_file(filename, overwrite=True)
            L2B_mie_file_paths_loaded.append(filename)
        except Exception as e:
            print(f"No data for the period {measurement_start}-{measurement_stop} available")
            print(e.__context__)
    else:
        L2B_mie_file_paths_loaded.append(filename)

# Check file list for empty files
L2B_mie_file_paths_valid = [i for i in L2B_mie_file_paths_loaded if nc.Dataset(i).groups]

**Read netCDF-data as xarray dataset (dask array)**  
xarray.open_mfdataset can be used to read multiple netCDF files into a single xarray dataset. Because data is stored in groups, they must be explicitly passed as parameter to the open_mfdataset method. 
The group names can be checked with the netCDF4 module and are depending on the requested fields.
The dimension which shall be used for concatenation, must also be defined. Data is then loaded as Dask arrays. (https://xarray.pydata.org/en/stable/user-guide/dask.html)

In [ ]:
ds_rayleigh = xr.open_mfdataset(
    L2B_rayleigh_file_paths_valid,
    group="rayleigh_wind_data",
    combine="nested",
    concat_dim="rayleigh_wind_data",
)
ds_mie = xr.open_mfdataset(
    L2B_mie_file_paths_valid, group="mie_wind_data", combine="nested", concat_dim="mie_wind_data"
)

### Load AUX_MET data
Define fields

In [ ]:
# Aeolus product
DATA_PRODUCT = "AUX_MET_12"

# off-nadir parameter
parameter_off_nadir = [
    "time",
    "latitude",
    "longitude",
    "surface_altitude",
    "layer_altitude",
    "layer_wind_component_u",
    "layer_wind_component_v",
]
parameter_off_nadir = [param + "_off_nadir" for param in parameter_off_nadir]

Generate file names and paths for saving data periods as netCDF files.  

In [ ]:
AUX_MET_filenames = [
    f"AUX_MET_{start.strftime('%Y-%m-%d')}-{end.strftime('%Y-%m-%d')}.nc"
    for start, end in zip(start_dates, end_dates)
]
user_folder = os.path.expanduser("~")
AUX_MET_folder = os.path.join(user_folder, "files/AUX_MET")
os.makedirs(AUX_MET_folder, exist_ok=True)
AUX_MET_file_paths = [os.path.join(AUX_MET_folder, filename) for filename in AUX_MET_filenames]

**Request AUX_MET data**  
This can take some time depending on the requested time period.

In [ ]:
# Create Aeolus request
request = AeolusRequest()
request.set_collection(DATA_PRODUCT)
request.set_fields(fields=parameter_off_nadir)

# Set bounding box for region around Germany
request.set_bbox({"n": 60, "w": 0, "s": 45, "e": 20})

# create list which will hold the loaded AUX_MET data
AUX_MET_file_paths_loaded = []
for measurement_start, measurement_stop, filename in zip(
    start_dates, end_dates, AUX_MET_file_paths
):
    # check if file already exists
    if not os.path.isfile(filename):
        try:
            AUX_MET_data = request.get_between(
                start_time=measurement_start,
                end_time=measurement_stop,
                filetype="nc",
                asynchronous=True,
            ).to_file(filename, overwrite=True)
            AUX_MET_file_paths_loaded.append(filename)
        except RuntimeError:
            print(f"No data for the period {measurement_start}-{measurement_stop} available")
    else:
        AUX_MET_file_paths_loaded.append(filename)

# Check file list for empty files (here if variables exist)
AUX_MET_file_paths_valid = [i for i in AUX_MET_file_paths_loaded if nc.Dataset(i).variables]

**Read netCDF-data as xarray dataset (dask array)**  
AUX_MET data is not saved in netCDF-groups and thus no group must be passed.

In [ ]:
ds_aux_met = xr.open_mfdataset(AUX_MET_file_paths, combine="nested", concat_dim="off_nadir")

## Spatial and temporal collocation (in progress)
AUX_MET data is available with 3 seconds temporal resolution and 137 altitude levels with around 100 levels up to 25 km. That is much higher than the Aeolus vertical resolution of 24 atmospheric range bins.  
In the next steps a temporal and spatially horizontal collocation is needed to look for all AUX_MET profiles within one L2B group.
Subsequently, a vertical averaging to the Aeolus range bin size is needed.  
The L2B line of sight azimuth angle must then be used to calculate the HLOS wind speeds from the wind components in the AUX_MET data.

## Data filtering (in progress)
Currently, using data from the FM-A data set still requires exclusion of hot-pixel data which will be solved with the coming reprocessed data set.
In addition, also blacklisted data has to be excluded.

## Determination of random and systematic errors (in progress)
